# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# path
cities_path="../WeatherPy/Output/cities_weather.csv"
# Read the cities data
cities_data= pd.read_csv(cities_path)
cities_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Vaini,-21.2000,-175.2000,29.00,84,75,5.66,TO,2021-02-07 00:24:06
1,Chunhuhub,19.5500,-88.6833,27.30,68,6,1.55,MX,2021-02-07 00:25:34
2,Punta Arenas,-53.1500,-70.9167,12.00,62,0,8.23,CL,2021-02-07 00:23:18
3,George,-33.9630,22.4617,19.00,94,75,3.60,ZA,2021-02-07 00:25:35
4,Torres,-29.3353,-49.7269,21.10,83,86,1.89,BR,2021-02-07 00:25:35
...,...,...,...,...,...,...,...,...,...
531,Srednekolymsk,67.4500,153.6833,-27.27,83,27,0.24,RU,2021-02-07 00:26:50
532,Kaduy,59.2000,37.1500,-15.89,93,100,4.01,RU,2021-02-07 00:26:51
533,Hokitika,-42.7167,170.9667,19.03,68,8,2.57,NZ,2021-02-07 00:26:51
534,Pingzhuang,42.0372,119.2889,-15.70,88,12,2.06,CN,2021-02-07 00:26:51


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#find the max humidity
city=[]
lat=[]
lng=[]
hum=[]
max =cities_data["Humidity"].max()
for index, row in cities_data.iterrows():
    if row["Humidity"] == max:
        city.append(row["City"])
        lat.append(row["Lat"])
        lng.append(row["Lng"])
        hum.append(row["Humidity"])
max_dic = {
    "City": city,
    "Lat": lat,
    "Lng": lng,
    "Humidity": hum
}
max_df = pd.DataFrame(max_dic)  
max_df
max_locations = max_df[["Lat", "Lng"]]

In [4]:
# Store latitude and longitude in locations
locations = cities_data[["Lat", "Lng"]]
# Fill NaN values and convert to float
humidity = cities_data["Humidity"].astype(float)
# Plot Heatmap
fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,max_intensity=30, point_radius=1)
max_heat= gmaps.symbol_layer(max_locations, fill_color='green', stroke_color='green', scale=3)
# Add layer
fig.add_layer(heat_layer)
fig.add_layer(max_heat)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.
* Store into variable
* Add a "Hotel Name" column to the DataFrame.

In [5]:
hotel_df=cities_data.loc[(cities_data['Max Temp']>28)&(cities_data['Max Temp']<80)&(cities_data['Wind Speed']<=15)&(cities_data['Cloudiness']==0)]
# Add columns for Country name, Hotel name, hotel rating
hotel_df=hotel_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Raiting"] = ""
hotel_df["Hotel Address"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Raiting,Hotel Address
0,Kathu,7.9178,98.3332,28.89,88,0,1.03,TH,2021-02-07 00:25:52,,,


### Hotel Map
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and print url
    name_address = requests.get(base_url, params=params)
    #print the name_address url, avoid doing for public github repos in order to avoid exposing key
    # convert to json
    name_address = name_address.json()
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
        hotel_df.loc[index, "Hotel Raiting"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_df.loc[index, "Hotel Name"] = np.nan
        hotel_df.loc[index, "Hotel Address"] = np.nan
        hotel_df.loc[index, "Hotel Raiting"] = np.nan

In [7]:
hotel_df=hotel_df.dropna(how='any')
hotel_df
# Save Data to csv
#cities_pd.to_csv("Airport_Output.csv")


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Raiting,Hotel Address
0,Kathu,7.9178,98.3332,28.89,88,0,1.03,TH,2021-02-07 00:25:52,Burasari Phuket,4.5,"18/110 Ruamjai Road Pa Tong, Amphur Kathu Phuket"


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
hotel_markers = gmaps.marker_layer(hotel_locations,info_box_content=hotel_info)
# Display figure
fig.add_layer(hotel_markers)
fig.add_layer (heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))